In [ ]:
%matplotlib inline
#
from LRGSG_package.shared import *
from LRGSG_package.config.utils import log_binning
from LRGSG_package.LRGSG import Lattice2D, Lattice3D, SignedRW, move_to_rootf
#
move_to_rootf()
plt.style.use("ipynb/nb_plotsheet.mplstyle")

In [ ]:
def process_signed_random_walk(data):
    """
    Processes a list of node-sign pairs to determine the distribution of unique nodes
    within clusters of consecutive identical signs.

    Args:
    data (list of tuples): List where each tuple contains a node (as a tuple) and a sign (+1 or -1).

    Returns:
    dict: A dictionary where keys are signs and values are lists of counts of unique nodes per cluster.
    """
    cluster_sizes = defaultdict(list)
    if not data:
        return dict(cluster_sizes)  # Return an empty dictionary if data is empty

    current_cluster = set()
    current_sign = data[0][1]

    for node, sign in data:
        if sign != current_sign:
            # Store size of the current cluster and reset
            cluster_sizes[current_sign].append(len(current_cluster))
            current_cluster = set()
            current_sign = sign
        current_cluster.add(node)

    # Add the last cluster
    cluster_sizes[current_sign].append(len(current_cluster))

    return dict(cluster_sizes)

In [ ]:
for pflip in [0.01, 0.06, 0.08, 0.103, 0.25, 0.5]:
    data = []
    for na in range(10):
        l = Lattice2D(10, pflip=pflip)
        l.flip_random_fract_edges()
        srw = SignedRW(l, simTime=2000, 
                initMode="random", 
                runMode="py",
                storeMode="sequential",
                storeFreq=1)
        srw.run()
        data.extend(srw.s_tList)
    cluster_sizes = process_signed_random_walk(data)
    edges, hist = powerlaw.pdf(cluster_sizes[-1], xmin=1)
    bin_centers = (edges[1:]+edges[:-1])/2.0
    plt.loglog(bin_centers, hist)
    edges, hist = powerlaw.pdf(cluster_sizes[1], xmin=1)
    bin_centers = (edges[1:]+edges[:-1])/2.0
    plt.loglog(bin_centers, hist)
    plt.show()

In [ ]:
fig, ax = plt.subplots()
l_ini = Lattice2D(32)
mmax = l_ini.Ne//10
for i in range(1, mmax, mmax//8):
    print("Current value of p:", i/l_ini.Ne)  # Debugging statement to check p
    l = Lattice2D(32, pflip=i/l_ini.Ne)
    l.flip_random_fract_edges()
    srw = SignedRW(l, simTime=2000, 
               initMode="random", 
               runMode="py",
               storeMode="sequential",
               storeFreq=1)
    srw.run()
    a = np.array(srw.s_tList, dtype=object)[:, 1].astype(int)
    changes = np.where(np.diff(a) != 0)[0] + 1
    sequence_lengths = np.diff(np.hstack(([0], changes, [len(a)])))
    # print(sequence_lengths)
    # b, h, w = log_binning(sequence_lengths, binnum=15)
    # ax.plot(b, h/w)
    edges, hist = powerlaw.pdf(sequence_lengths, xmin=1)
    bin_centers = (edges[1:]+edges[:-1])/2.0
    ax.loglog(bin_centers, hist)
plt.show()

In [ ]:
fig, ax = plt.subplots()
for p in [0.01, 0.05, 0.08, 0.1, 0.2, 0.5]:
    print("Current value of p:", p)  # Debugging statement to check p
    l = Lattice2D(128, pflip=p)
    l.flip_random_fract_edges()
    srw = SignedRW(l, simTime=2000, 
               initMode="random", 
               runMode="py",
               storeMode="sequential",
               storeFreq=1)
    srw.run()
    a = np.array(srw.s_tList, dtype=object)[:, 1].astype(int)
    changes = np.where(np.diff(a) != 0)[0] + 1
    sequence_lengths = np.diff(np.hstack(([0], changes, [len(a)])))
    print(sequence_lengths)
    # b, h, w = log_binning(sequence_lengths, binnum=15)
    # ax.plot(b, h/w)
    edges, hist = powerlaw.pdf(sequence_lengths)
    bin_centers = (edges[1:]+edges[:-1])/2.0
    ax.loglog(bin_centers, hist)
plt.show()

In [ ]:
b, h, w = log_binning(sequence_lengths, binnum=15)
plt.plot(b, h/w)
plt.yscale("log")
plt.xscale("log")

In [ ]:
a.alpha